## ANALISIS RETIRO AL 12/2018

Por la forma en la que las compañías presentaron la información y por los puntos que se desea analizar es que no se puede crear un dataset unificado.

Se procede a generar lo siguiente:
       * Dataset por Póliza: Se obtendrá por cia, poliza y causante la suma de la renta liquidada, renta garantizada y las reservas. Aquí no pueden haber registros duplicados respecto a cia, poliza y causante.
       * Dataset por Beneficiario: Se obtendrá por cia la renta liquidada y los datos de los beneficiarios. Aquí necesariamente cia, poliza y causante se repetirán por la cantidad de beneficiarios que tenga cada póliza.
       
Asimismo, como tampoco las cias expusieron la información de manera homogenea entre ellas, para obtener ambos datasets se tendrán que procesar por grupos.

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time
import datetime

%matplotlib inline

#pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (7, 5)

#### GRUPO 1: BINARIA, CREDICOOP, EXPERTA, HSBC, IAPSER, LA SEGUNDA, METLIFE, NACION

In [2]:
fullpath = "C:/Users/eangenello/Desktop/TECNICA/Python/input/RETIRO/Dataset_Retiro.xlsx"

binaria=pd.read_excel(fullpath, sheet_name = "BINARIA")
credicoop=pd.read_excel(fullpath, sheet_name = "CREDICOOP")
experta=pd.read_excel(fullpath, sheet_name = "EXPERTA")
hsbc=pd.read_excel(fullpath, sheet_name = "HSBC")
iapser=pd.read_excel(fullpath, sheet_name = "IAPSER")
la_segunda=pd.read_excel(fullpath, sheet_name = "LA SEGUNDA")
metlife=pd.read_excel(fullpath, sheet_name = "METLIFE")
nacion=pd.read_excel(fullpath, sheet_name = "NACION")

In [3]:
data1 = binaria.append(credicoop)
data1 = data1.append(experta)
data1 = data1.append(hsbc)
data1 = data1.append(iapser)
data1 = data1.append(la_segunda)
data1 = data1.append(metlife)
data1 = data1.append(nacion)

data1['Cuil Causante'] = data1['Cuil Causante'].astype(object)
data1['Cuil Beneficiario'] = data1['Cuil Beneficiario'].astype(object)
data1['Nro. Póliza'] = data1['Nro. Póliza'].astype(object)
data1['Nro. Beneficio'] = data1['Nro. Beneficio'].astype(object)

#Copio los datos para usarlos para los dos distintos datasets
data1_poliza=data1.copy()
data1_beneficiario=data1.copy()

#### GRUPO 2: PROYECCION, GALICIA, ORIGENES, SAN CRISTOBAL, SMG

In [4]:
fullpath = "C:/Users/eangenello/Desktop/TECNICA/Python/input/RETIRO/Dataset_Retiro.xlsx"

proyeccion=pd.read_excel(fullpath, sheet_name = "PROYECCION")
galicia=pd.read_excel(fullpath, sheet_name = "GALICIA")
origenes=pd.read_excel(fullpath, sheet_name = "ORIGENES")
san_cristobal=pd.read_excel(fullpath, sheet_name = "SAN CRISTOBAL")
smg=pd.read_excel(fullpath, sheet_name = "SMG")

In [5]:
data2 = proyeccion.append(galicia)
data2 = data2.append(origenes)
data2 = data2.append(san_cristobal)
data2 = data2.append(smg)

data2['Cuil Causante'] = data2['Cuil Causante'].astype(object)
data2['Cuil Beneficiario'] = data2['Cuil Beneficiario'].astype(object)
data2['Nro. Póliza'] = data2['Nro. Póliza'].astype(object)
data2['Nro. Beneficio'] = data2['Nro. Beneficio'].astype(object)

#Copio los datos para usarlos para los dos distintos datasets
data2_poliza=data2.copy()
data2_beneficiario=data2.copy()

### Dataset por póliza

#### GRUPO 1

In [6]:
#Sumo las rentas liquidadas y garantizadas
data1_poliza_agrup=data1_poliza.groupby(['Entidad','Nro. Póliza','Cuil Causante'], as_index=False)['Renta Liq.','Renta Gar.'].sum() 

data1_poliza_agrup['Cuil Causante'] = data1_poliza_agrup['Cuil Causante'].astype(object)
data1_poliza_agrup['Nro. Póliza'] = data1_poliza_agrup['Nro. Póliza'].astype(object)

In [7]:
# Del dataset original le elimino las columnas que agrupé previamente

data1_poliza_solo= data1_poliza.drop(['Renta Liq.', 'Renta Gar.'],1)

In [8]:
# Elimino los registris duplicados respecto a entidad, poliza y causante

data1_poliza_solo = data1_poliza_solo.drop_duplicates(['Entidad','Nro. Póliza','Cuil Causante'], keep='first')

In [9]:
# Agrupo ambas tablas

data1_poliza = data1_poliza_agrup.merge(data1_poliza_solo, how='left', on=['Entidad','Nro. Póliza','Cuil Causante']) 

In [10]:
# Reordeno

data1_poliza = data1_poliza[['Entidad', 'Nro. Póliza', 'Cuil Causante','Inicio de Vigencia', 'Tipo de Renta', 
                             'Prestacion', 'Renta Liq.', 'Renta Gar.', 'Reserva',  'Ajuste']]

#### GRUPO 2

In [11]:
#Sumo sólo las rentas liquidadas
data2_poliza_agrup=data2_poliza.groupby(['Entidad','Nro. Póliza','Cuil Causante'], as_index=False)['Renta Liq.'].sum() 

data2_poliza_agrup['Cuil Causante'] = data2_poliza_agrup['Cuil Causante'].astype(object)
data2_poliza_agrup['Nro. Póliza'] = data2_poliza_agrup['Nro. Póliza'].astype(object)

In [12]:
# Del dataset original le elimino la columna que agrupé previamente

data2_poliza_solo= data2_poliza.drop(['Renta Liq.'],1)

In [13]:
# Elimino los registris duplicados respecto a entidad, poliza y causante

data2_poliza_solo = data2_poliza_solo.drop_duplicates(['Entidad','Nro. Póliza','Cuil Causante'], keep='first')

In [14]:
# Agrupo ambas tablas

data2_poliza = data2_poliza_agrup.merge(data2_poliza_solo, how='left', on=['Entidad','Nro. Póliza','Cuil Causante']) 

In [15]:
# Reordeno

data2_poliza = data2_poliza[['Entidad', 'Nro. Póliza', 'Cuil Causante','Inicio de Vigencia', 'Tipo de Renta', 
                             'Prestacion', 'Renta Liq.', 'Renta Gar.', 'Reserva',  'Ajuste']]

In [16]:
data_poliza = data1_poliza.append(data2_poliza)
data_poliza.head()

,Entidad,Nro. Póliza,Cuil Causante,Inicio de Vigencia,Tipo de Renta,Prestacion,Renta Liq.,Renta Gar.,Reserva,Ajuste
0,BINARIA,1,"20,045,231,373.00",1995-12-01 00:00:00,RVP,PF,"2,640.83","2,640.83","313,504.45",TT
1,BINARIA,2,"23,087,065,049.00",1995-12-01 00:00:00,RVP,PF,"2,000.87","2,000.87","279,513.71",TT
2,BINARIA,3,"20,068,998,280.00",1995-12-01 00:00:00,RVP,PF,"1,755.19","1,755.19","191,574.66",TT
3,BINARIA,4,"20,119,977,526.00",1995-12-01 00:00:00,RVP,PF,"1,421.85","1,421.85","311,100.87",TT
4,BINARIA,5,"20,049,739,789.00",1995-12-01 00:00:00,RVP,PF,"1,451.13","1,451.13","201,046.98",TT


In [17]:
data_poliza.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\data_poliza.xlsx', sheet_name='data_poliza', index=False)

### Dataset por beneficiario

In [18]:
data_beneficiario=data1_beneficiario.append(data2_beneficiario)
data_beneficiario.columns

Index(['Cuil Causante', 'Apellido', 'Nombres', 'Nro. Póliza',
       'Inicio de Vigencia', 'Tipo de Renta', 'Prestacion', 'Renta Liq.',
       'Renta Gar.', 'Reserva', 'Ajuste', 'Anses', 'Nro. Beneficio',
       'Nro. Beneficiarios', 'Cuil Beneficiario', 'Tipo Beneficiario',
       '% Beneficio', 'Sexo', 'Fecha de Nac.', 'Entidad'],
      dtype='object')

In [19]:
data_beneficiario=data_beneficiario[['Entidad', 'Nro. Póliza','Cuil Causante', 'Apellido', 'Nombres', 'Inicio de Vigencia',  
                                    'Tipo de Renta', 'Prestacion', 'Renta Liq.',  'Ajuste', 'Anses', 'Nro. Beneficio', 
                                    'Cuil Beneficiario', 'Tipo Beneficiario', '% Beneficio', 'Sexo', 'Fecha de Nac.']]

In [20]:
data_beneficiario.head()

,Entidad,Nro. Póliza,Cuil Causante,Apellido,Nombres,Inicio de Vigencia,Tipo de Renta,Prestacion,Renta Liq.,Ajuste,Anses,Nro. Beneficio,Cuil Beneficiario,Tipo Beneficiario,% Beneficio,Sexo,Fecha de Nac.
0,BINARIA,1,"20,045,231,373.00",GIMENEZ ARROYO,HORACIO FELIPE,1995-12-01 00:00:00,RVP,PF,"2,640.83",TT,SI,"15,580,135,340.00","27,049,831,388.00",C,0.70,F,1944-10-08 00:00:00
1,BINARIA,2,"23,087,065,049.00",PEREZ,MILCIADES AMERICO,1995-12-01 00:00:00,RVP,PF,"2,000.87",TT,NO,NaN,"27,057,941,478.00",C,0.70,F,1948-10-29 00:00:00
2,BINARIA,3,"20,068,998,280.00",LEMOS,ADOLFO CESAR,1995-12-01 00:00:00,RVP,PF,"1,755.19",TT,NO,NaN,"27,045,629,495.00",C,0.70,F,1942-12-21 00:00:00
3,BINARIA,4,"20,119,977,526.00",CALLEJAS,RICARDO ALFREDO,1995-12-01 00:00:00,RVP,PF,"1,421.85",TT,SI,"15,580,161,180.00","27,170,686,980.00",C,0.70,F,1965-11-16 00:00:00
4,BINARIA,5,"20,049,739,789.00",BERSI,ATILIO EDUARDO,1995-12-01 00:00:00,RVP,PF,"1,451.13",TT,NO,NaN,"27,057,643,493.00",C,0.70,F,1948-08-15 00:00:00


In [21]:
data_beneficiario.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\data_beneficiario.xlsx', sheet_name='data_beneficiario', index=False)